In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

### add command path

In [2]:
import sys,numpy,pandas,os
from copy import deepcopy
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')

### import functions

In [28]:
from command.nexus.SPheno import SPheno,block_mapping
from command.scan.scan import scan
from command.multi_thread.MT_SPheno import MT_SPheno
from command.multi_thread.queue_operation import GenerateQueue # generate random queue
from command.pytorch.normalize import normalize
from command.operators.iterable import FlatToList
from command.data_transformer.ArrayToInputQueue import ArrayToInputQueue
from command.data_transformer.InputToPandas import InputToPandas

In [4]:
mold=scan(method='random')
# MinPar
mold.AddScalar('tanB','MINPAR',3,1.,60.)
# Extpar
mold.AddScalar('Lambda','EXTPAR'  ,61  ,1e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('Kappa','EXTPAR'   ,62 ,1.e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('A_Lambda','EXTPAR' ,63,-3.e3,3.e3)
mold.AddScalar('A_kappa','EXTPAR' ,64,-3.e3,3.e3)
mold.AddScalar('mu_eff','EXTPAR'  ,65,100.,1500.)
# Squark mass
mold.AddElement( 'Mq32','MSQ2',(3,3),1e4,4e6)
mold.AddElement( 'Mu32','MSU2',(3,3),1e4,4e6)
mold.AddFollower('Md32','MSD2',(3,3),'Mu32')
# Slepton mass
mold.AddElement( 'Ml32','MSL2',(3,3),1e4,4e6)
mold.AddFollower('Ml22','MSL2',(2,2),'Ml32')
mold.AddFollower('Ml12','MSL2',(1,1),'Ml32')
mold.AddFollower('Me32','MSE2',(3,3),'Ml32')
mold.AddFollower('Me22','MSE2',(2,2),'Ml32')
mold.AddFollower('Me12','MSE2',(1,1),'Ml32')
# Yukawa
mold.AddElement( 'Tt' ,'TU'  ,(3,3),-2000,2000)
mold.AddElement( 'Tb' ,'TD'  ,(3,3),-2000,2000)
mold.AddElement( 'Ttau' ,'TE'  ,(3,3),-2000,2000)
# MSoft
mold.AddScalar('M1','MSOFT',1,50,2000)
mold.AddScalar('M2','MSOFT',2,100,2000)

In [5]:
mold.GetValue('LesHouches.in.NMSSM_low',mapping=block_mapping)
mold.Print()

     Kappa: 0.11
        Tt: 1000.0
      Ml22: 40000.0
  A_Lambda: -1500.0
        M2: 2000.0
        Tb: 1000.0
   A_kappa: -36.0
      Ttau: 0.0
      Me32: 40000.0
      Me12: 40000.0
      Mq32: 1000000.0
      Ml32: 40000.0
      Me22: 40000.0
      Md32: 1000000.0
    Lambda: 0.1
        M1: 500.0
    mu_eff: 965.0
      Mu32: 1000000.0
      tanB: 10.0
      Ml12: 40000.0


In [6]:
order=FlatToList(map(lambda x: list(x.keys()),mold.free_parameter_list.maps))
order

['tanB',
 'Lambda',
 'Kappa',
 'A_Lambda',
 'A_kappa',
 'mu_eff',
 'M1',
 'M2',
 'Mq32',
 'Mu32',
 'Ml32',
 'Tt',
 'Tb',
 'Ttau']

In [7]:
dimention=len(order)
dimention

14

### normalize & tabulate parameters from mold list to numpy array

In [8]:
Norm=normalize(mold,order=order) # set operation scale=(a,b) to scale parameters to (a,b) range
Norm.scale

(0, 1)

### make a input point queue
*choose one*

In [8]:
# Random points queue by scan's function: Sample
RandSamples=GenerateQueue(mold,lenth=1000) 

In [19]:
# # Random points queue from numpy.random array
# normal_array=numpy.random.rand(1000,dimention) 
# par_array=Norm.D(normal_array)
# RandSamples=ArrayToInputQueue(par_array,mold,order=order)

### multithread SPheno

In [9]:
MTS=MT_SPheno(threads=6,Renew=False)

SPheno file:
->/home/heyangle/Desktop/ScanCraft/ScanCraft/packages/SPheno-4.0.3


In [9]:
MTS.Run(RandSamples,report_interval=1000)#,ReMake='NMSSM_sarah')

Calculations begin at Tue Jun 26 17:05:34 2018
  threads:	6
  points:	1000
Thread-3 got 0 samples
Thread-0 got 0 samples
Thread-2 got 0 samples
Thread-5 got 0 samples
Thread-4 got 0 samples
Thread-1 got 0 samples
  thread-5	runing,        0 points left at Tue Jun 26 17:16:17 2018
All points done. Use 0.183557 hours


KeyError: 'spectrum'

*# read from record*

In [13]:
output_dir=MTS.harvest_dir
os.listdir(output_dir)

['record_180626_171635']

In [52]:
# from command.file_operations.GetSamples import GetSamples
# samples=GetSamples(path='./output/record_180626_171635/',patterns=['out'])
# spectr_list=[]
# for sample in samples:
#     if 'out' in sample.documents:
#         sample.Merge(SPheno.Read(None,sample.documents['out']))
#         spectr_list.append(sample)

168

### Chisqure

In [57]:
def Chisqure(sample):
    X2_h=(sample.MASS[25]-125.01)**2/9
    return X2_h

In [69]:
X2s=numpy.array(list(map(Chisqure,spectr_list)))

In [70]:
list(zip([s.MASS[25] for s in spectr_list],X2s))

[(113.68902, 14.240509795600016),
 (110.81493, 22.3888902561),
 (122.479057, 0.7117413854721154),
 (113.850167, 13.837985843098792),
 (117.483461, 6.29430992428011),
 (113.515208, 14.68113812480712),
 (112.641666, 16.997298437284012),
 (93.6808613, 109.05721463153753),
 (116.571046, 7.912882734901796),
 (112.393885, 17.6851508548028),
 (117.36887, 6.487429741877785),
 (106.16894, 39.442837991511105),
 (115.951773, 9.116830709281004),
 (117.257381, 6.678122373240128),
 (112.971105, 16.103888091225027),
 (105.454059, 42.492758710609024),
 (106.911123, 36.39659429568101),
 (119.286102, 3.640334257156007),
 (116.985795, 7.154207320225017),
 (113.803374, 13.954274033764001),
 (112.873137, 16.367049275641012),
 (120.3635, 2.398884694444448),
 (118.514014, 4.688648234688447),
 (116.139941, 8.741994073720134),
 (105.176065, 43.70944195380283),
 (116.194413, 8.634952683841014),
 (108.109336, 31.7369381823218),
 (115.792635, 9.439979727025001),
 (117.374421, 6.478007407249012),
 (112.722813, 16.

In [72]:
X2s.reshape(-1,1)

array([[ 14.2405098 ],
       [ 22.38889026],
       [  0.71174139],
       [ 13.83798584],
       [  6.29430992],
       [ 14.68113812],
       [ 16.99729844],
       [109.05721463],
       [  7.91288273],
       [ 17.68515085],
       [  6.48742974],
       [ 39.44283799],
       [  9.11683071],
       [  6.67812237],
       [ 16.10388809],
       [ 42.49275871],
       [ 36.3965943 ],
       [  3.64033426],
       [  7.15420732],
       [ 13.95427403],
       [ 16.36704928],
       [  2.39888469],
       [  4.68864823],
       [  8.74199407],
       [ 43.70944195],
       [  8.63495268],
       [ 31.73693818],
       [  9.43997973],
       [  6.47800741],
       [ 16.77499604],
       [  8.98309195],
       [ 26.93590624],
       [ 20.2141329 ],
       [  7.77707079],
       [  3.25270724],
       [  1.41335164],
       [  9.98201899],
       [ 12.95134465],
       [ 15.2607709 ],
       [ 63.70877472],
       [ 21.62811136],
       [ 11.49194632],
       [ 24.21223219],
       [ 68